In [1]:
from torchvision import transforms
from torchvision.utils import Image
import os
import torch.nn as nn
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
import tensorflow as tf
import torch
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.optim as optim
import matplotlib.pyplot as plt
import shutil
import cv2
import numpy as np

2024-03-25 23:50:23.561293: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 23:50:23.596489: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 23:50:23.596515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 23:50:23.597225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 23:50:23.603719: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def Read_Data():
  temp = []
  temp2 = []
  for subs in os.listdir(f"dataset/images/train"):
    temp.append(f"dataset/images/train/{subs}")
  for subs in os.listdir(f"dataset/masks/train"):
    temp2.append(f"dataset/masks/train/{subs}")
  result = [list(pair) for pair in zip(temp, temp2)]
  return result

In [3]:
train= Read_Data()

In [4]:
train_x = []
train_y = []
transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])
for i in train:
    mask = Image.open(i[0])
    mask_tensor = transform(mask)
    mask_tensor.to("cuda")
    train_x.append(mask_tensor)

for i in train:
    mask = Image.open(f"{i[1]}")
    mask = transform(mask)
    mask_tensor.to("cuda")
    binary_mask = torch.where(mask == 1/255, torch.tensor(1.0), torch.tensor(0.0))
    train_y.append(binary_mask)

In [5]:
print(train_x[0])

tensor([[[0.5137, 0.5176, 0.5216,  ..., 0.7176, 0.7137, 0.7137],
         [0.5176, 0.5216, 0.5255,  ..., 0.7216, 0.7176, 0.7176],
         [0.5216, 0.5255, 0.5294,  ..., 0.7216, 0.7176, 0.7176],
         ...,
         [0.0275, 0.0353, 0.0471,  ..., 0.0471, 0.0510, 0.0549],
         [0.0275, 0.0392, 0.0431,  ..., 0.0627, 0.0627, 0.0667],
         [0.0235, 0.0314, 0.0431,  ..., 0.0549, 0.0588, 0.0588]],

        [[0.7569, 0.7608, 0.7686,  ..., 0.9137, 0.9098, 0.9098],
         [0.7608, 0.7647, 0.7725,  ..., 0.9176, 0.9137, 0.9137],
         [0.7647, 0.7686, 0.7725,  ..., 0.9176, 0.9137, 0.9137],
         ...,
         [0.0549, 0.0549, 0.0588,  ..., 0.1216, 0.1216, 0.1255],
         [0.0627, 0.0627, 0.0588,  ..., 0.1373, 0.1333, 0.1373],
         [0.0627, 0.0588, 0.0627,  ..., 0.1373, 0.1255, 0.1255]],

        [[0.8157, 0.8196, 0.8275,  ..., 0.9412, 0.9451, 0.9451],
         [0.8196, 0.8235, 0.8314,  ..., 0.9451, 0.9490, 0.9490],
         [0.8235, 0.8275, 0.8314,  ..., 0.9490, 0.9490, 0.

In [6]:
data_set = []
for i in range(len(train_x)):
    data_set.append([train_x[i],train_y[i]])

In [11]:
dataset = DataLoader(data_set, batch_size=16, shuffle=True)

In [12]:
class UNetWithResNet(nn.Module):
    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels,kernel_size=1),
            nn.ReLU(inplace=True)

        )
    def __init__(self):
        super(UNetWithResNet, self).__init__()
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.enc5 = self.conv_block(512, 1024)
        self.max = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(1024, 512)
        self.dec2 = self.conv_block(512, 256)
        self.dec3 = self.conv_block(256, 128)
        self.dec4 = self.conv_block(128, 64)
        self.out = nn.Conv2d(64, 1, kernel_size=1)
        
    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(self.max(x1))
        x3 = self.enc3(self.max(x2))
        x4 = self.enc4(self.max(x3))
        x5 = self.enc5(self.max(x4))
        
        x = self.upconv4(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.dec1(x)

        x = self.upconv3(x)
        x = torch.cat([x, x3], dim=1)
        x = self.dec2(x)

        x = self.upconv2(x)
        x = torch.cat([x, x2], dim=1)
        x = self.dec3(x)

        x = self.upconv1(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec4(x)
        return self.out(x)

In [13]:
model = UNetWithResNet()
model.to("cuda")
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in dataset:
        optimizer.zero_grad()
        outputs = model(images.to("cuda"))
        loss = criterion(outputs, targets.to("cuda"))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/100], Loss: 0.7178
Epoch [2/100], Loss: 0.6970
Epoch [3/100], Loss: 0.6924
Epoch [4/100], Loss: 0.6778
Epoch [5/100], Loss: 0.6832
Epoch [6/100], Loss: 0.6719
Epoch [7/100], Loss: 0.6778
Epoch [8/100], Loss: 0.6659
Epoch [9/100], Loss: 0.6685
Epoch [10/100], Loss: 0.6661
Epoch [11/100], Loss: 0.6836
Epoch [12/100], Loss: 0.6769
Epoch [13/100], Loss: 0.6684
Epoch [14/100], Loss: 0.6700
Epoch [15/100], Loss: 0.6588
Epoch [16/100], Loss: 0.6503
Epoch [17/100], Loss: 0.6639
Epoch [18/100], Loss: 0.6593
Epoch [19/100], Loss: 0.6559
Epoch [20/100], Loss: 0.6592
Epoch [21/100], Loss: 0.6604
Epoch [22/100], Loss: 0.6633
Epoch [23/100], Loss: 0.6506
Epoch [24/100], Loss: 0.6682
Epoch [25/100], Loss: 0.6536
Epoch [26/100], Loss: 0.6505
Epoch [27/100], Loss: 0.6758
Epoch [28/100], Loss: 0.6483
Epoch [29/100], Loss: 0.6750
Epoch [30/100], Loss: 0.6555
Epoch [31/100], Loss: 0.6607
Epoch [32/100], Loss: 0.6529
Epoch [33/100], Loss: 0.6641
Epoch [34/100], Loss: 0.6608
Epoch [35/100], Loss: 0

In [17]:
torch.save(model.state_dict(), 'f2_model.pt')

In [18]:
model = UNetWithResNet()
state_dic = torch.load('f2_model.pt')
model.load_state_dict(state_dic)

<All keys matched successfully>

In [20]:
img = Image.open("dog1.jpg")
transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])
transform2 = transforms.ToPILImage()
trans_img = transform(img)
result = model(trans_img)
plt.imshow(transform2(result))
plt.show()

RuntimeError: Given groups=1, weight of size [512, 1024, 1, 1], expected input[1, 512, 64, 32] to have 1024 channels, but got 512 channels instead